Could the non-ML pYIN algorithm be faster than CREPE? It depends on the parameters.

In [ ]:
from ddsp.spectral_ops import compute_f0
from thesis.notebook_util import specplot, play_audio
import librosa
import numpy as np

In [ ]:
import ddsp.training.data
data_provider = ddsp.training.data.TFRecordProvider(
#     "../data/datasets/violin3/violin3.tfrecord*",
    "/home/vaclav/datasets/violin3/violin3.tfrecord*",
    frame_rate=50,
    centered=True,
    with_jukebox=False,
)
dataset = data_provider.get_batch(batch_size=1, shuffle=False, repeats=1)
dataset_iter = iter(dataset)

In [ ]:
for i in range(300):
    batch = next(dataset_iter)

In [ ]:
batch = next(dataset_iter)
audio = batch["audio_16k"]
audio = audio[:,:16000]
play_audio(audio)

In [ ]:
%%timeit -n 1 -r 100
f0_hz_crepe, f0_confidence_crepe = compute_f0(audio[0], frame_rate=50, model_name="crepe-tiny")

In [ ]:
%%timeit -n 1 -r 100
f0_hz_pyin, _, f0_confidence_pyin = librosa.pyin(
    audio[0].numpy(),
    fmin=librosa.note_to_hz('C2'),
    fmax=librosa.note_to_hz('C7'),
    sr=16000,
    frame_length=(16000//50)*4,
    resolution=0.2,
)

In [ ]:
%%timeit -n 1 -r 100
f0_hz_yin = librosa.yin(
    audio[0].numpy(),
    fmin=librosa.note_to_hz('C2'),
    fmax=librosa.note_to_hz('C7'),
    sr=16000,
    frame_length=(16000//50)*4,
)

In [ ]:
%%timeit
frequencies_piptrack, magnitudes_piptrack = librosa.piptrack(
    y=audio[0].numpy(),
    fmin=librosa.note_to_hz('C2'),
    fmax=librosa.note_to_hz('C7'),
    sr=16000,
    # frame_length=(16000//50)*4,
    hop_length=(16000//50),
)

index = magnitudes_piptrack.argmax(axis=0)

f0_hz_piptrack = np.array([f[i] for f, i in zip(frequencies_piptrack.T, index)])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(f0_hz_crepe)
plt.plot(f0_hz_pyin)
plt.plot(f0_hz_yin)
plt.plot(f0_hz_piptrack)

In [ ]:
specplot(audio[0])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(f0_confidence_crepe[:50])
plt.plot(f0_confidence_pyin[:50])